#### Import modules

In [ ]:
import requests
import json
import copy
import csv

#### Get 17,034 protein chains from the non-redundant dataset at 40% sequence identity and store it as dictionary named 'data' with key as pdbname and value as sequence

In [ ]:
i_path = "/home/irfan/irfan_thesis/wt_wt_analysis/inputs"
o_path = "/home/irfan/irfan_thesis/wt_wt_analysis/outputs"

with open(f"{i_path}/cullpdb_pc40.0_res0.0-2.5_noBrks_len40-10000_R0.3_Xray_d2021_11_19_chains17034.fasta.txt",'r') as f:
    lines = f.readlines()
    
data = {}
i = 0
while True:
    try:
        line = lines[i].rstrip()
    except:
        break
    
    if line.startswith(">"):
        name, seq = line, []
        i+=1
        
        line = lines[i].rstrip()
        while not line.startswith(">"):
            seq.append(line)
            i+=1
            try:
                line = lines[i].rstrip()
            except:
                break
        
        seq = ''.join(seq)
        data[name[1:6]] = seq

#### store pdbnames as list named 'data_keys'

In [ ]:
data_keys = [i for i in data]

#### define funcion for getting taxonomic id from a given pdb chain

In [ ]:
def get_tax_id(pdb_i):
    pdb_id   = pdb_i[:4]
    chain_id = pdb_i[-1]

    query_1 = """
    {
      polymer_entity_instance(entry_id: None, asym_id: None) {
        rcsb_polymer_entity_instance_container_identifiers {
          entity_id
        }
      }
    }
    """

    query_1 = query_1.replace('entry_id: None', f'entry_id: "{pdb_id}"')
    query_1 = query_1.replace('asym_id: None', f'asym_id: "{chain_id}"')

    params_1 = (
        ('query',query_1),
    )


    response_1 = requests.get('https://data.rcsb.org/graphql', params=params_1)
    entity_id  = response_1.json()["data"]["polymer_entity_instance"]["rcsb_polymer_entity_instance_container_identifiers"]["entity_id"]

    query_2 = """
    {
      polymer_entity(entry_id: None, entity_id: None) {
        rcsb_entity_source_organism {
          beg_seq_num
          common_name
          end_seq_num
          ncbi_parent_scientific_name
          ncbi_scientific_name
          ncbi_taxonomy_id
          provenance_source
          scientific_name
          source_type
        }
      }
    }
    """

    query_2 = query_2.replace('entry_id: None', f'entry_id: "{pdb_id}"')
    query_2 = query_2.replace('entity_id: None', f'entity_id: "{entity_id}"' )

    params_2 = (
        ('query',query_2),
    )

    response_2  = requests.get('https://data.rcsb.org/graphql', params=params_2)
    taxonomy_id = response_2.json()["data"]["polymer_entity"]["rcsb_entity_source_organism"][0]["ncbi_taxonomy_id"]
    
    return taxonomy_id

#### retrieve multiple PDB chains having 95% sequence identity and same taxonomic ID as that of a given query PDB chain and store the output as csv file named 'q_pdb_to_t_pdbs_0.95.csv'  with query_pdb, taxonomy_id and pdb_ids as 1, 2 and 3 columns

In [ ]:
for i in range(0,len(data)):
    
    query_pdb           = data_keys[i]
    query_seq           = data[query_pdb]
    
    try:
        taxonomy_id         = str(get_tax_id(query_pdb))
    except:
        continue
    
    query = {
      "query": {
        "type": "group",
        "logical_operator": "and",
        "nodes": [
          {
        "type": "terminal",
        "service": "sequence",
        "parameters": {
          "target": "pdb_protein_sequence",
          "value": "",
          "identity_cutoff": 0.95,
          "evalue_cutoff": 0.1
        }
      },
          {
            "type": "terminal",
            "service": "text",
            "parameters": {
              "operator": "exact_match",
              "value": "",
              "attribute": "rcsb_entity_source_organism.taxonomy_lineage.id"
            }
          }
        ]
      },
      "return_type": "polymer_instance",
      "request_options": {
        "pager": {
          "start": 0,
          "rows": 100
        },
        "scoring_strategy": "combined",
        "sort": [
          {
            "sort_by": "score",
            "direction": "desc"
          }
        ]
      }
    }
    
    query["query"]["nodes"][0]["parameters"]["value"] = query_seq
    query["query"]["nodes"][1]["parameters"]["value"] = taxonomy_id
    query = json.dumps(query)
    
    params = (
        ('json',query),
    )
    
    while True:
        try:
            response = requests.get('https://search.rcsb.org/rcsbsearch/v1/query', params=params)
            status   = response.status_code 
            break
        except:
            continue

    pdb_ids = []
    if status == 200:

        results  = response.json()["result_set"]
        for result in results:
            pdb_id = result["identifier"]
            pdb_ids.append(pdb_id)
            
    pdb_ids = ", ".join(pdb_ids)
    
    with open(f'{o_path}/q_pdb_to_t_pdbs_0.95.csv', 'a', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([query_pdb, taxonomy_id, pdb_ids])
    
    print(i,status)